In [ ]:
#importing libraries
import os
import random
import glob
import pandas as pd
import numpy as np
import pydicom 
from torch.hub import load_state_dict_from_url
from torchvision.models.resnet import ResNet, Bottleneck
import torch
import torch.optim as optim
from albumentations import Compose, ShiftScaleRotate, Resize, Normalize, HorizontalFlip, RandomBrightnessContrast,CenterCrop
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset,Subset
import cv2
#from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import roc_auc_score, classification_report, recall_score, f1_score, accuracy_score, precision_score, jaccard_score
from tqdm import notebook as tqdm

from matplotlib import pyplot as plt

In [ ]:
!pip install opencv-python

In [ ]:
dir_csv = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection'

train_images_dir = '../input/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x'
test_images_dir = '../input/rsna-test-stage-1-images-png-224x/stage_1_test_png_224x'
# train_metadata_csv = '/home/navneeth/ICH_Code/input/train_metadata_noidx.csv'
# test_metadata_csv = '/home/navneeth/ICH_Code/input/test_metadata_noidx.csv'

In [ ]:
#PARAMS
n_classes = 6
n_epochs = 10
batch_size = 32

COLS = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']

In [ ]:
train = pd.read_csv(os.path.join(dir_csv, 'stage_2_train.csv'))
test = pd.read_csv(os.path.join(dir_csv, 'stage_2_sample_submission.csv'))

In [ ]:
class IntracranialDataset(Dataset):

    def __init__(self, csv_file, path, labels, transform=None):
        
        self.path = path
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.labels = labels

    def __len__(self):
        
        return len(self.data)

    def __getitem__(self, idx):
        
        img_name = os.path.join(self.path, self.data.loc[idx, 'Image'] + '.png')
        img = cv2.imread(img_name)
        
        if self.transform:       
            
            augmented = self.transform(image=img)
            img = augmented['image']   
            
        if self.labels:
            
            labels = torch.tensor(
                self.data.loc[idx, ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']])
            return {'image': img, 'labels': labels}    
        
        else:      
            
            return {'image': img}
    

In [ ]:
#Preparing data
train[['ID', 'Image', 'Diagnosis']] = train['ID'].str.split('_', expand=True)
train = train[['Image', 'Diagnosis', 'Label']]
train.drop_duplicates(inplace=True)
train = train.pivot(index='Image', columns='Diagnosis', values='Label').reset_index()
train['Image'] = 'ID_' + train['Image']

#remove images that are not saved properly as png 
png = glob.glob(os.path.join(train_images_dir, '*.png'))
png = [os.path.basename(png)[:-4] for png in png]
png = np.array(png)


train = train[train['Image'].isin(png)]
train.to_csv('train.csv',index=False)

In [ ]:
test[['ID','Image','Diagnosis']] = test['ID'].str.split('_', expand=True)

In [ ]:
test

In [ ]:
test['Image'] = 'ID_' + test['Image']

In [ ]:
test

In [ ]:
test = test[['Image', 'Label']]
test.drop_duplicates(inplace=True)

In [ ]:
test.head()

In [ ]:
png = glob.glob(os.path.join(test_images_dir, '*.png'))
png = [os.path.basename(png)[:-4] for png in png]
png = np.array(png)

test = test[test['Image'].isin(png)]

In [ ]:
test.head()
print(len(test))

No instances of Image IDs in csv found in image dataset

In [ ]:
# Prepare test table
test.to_csv('test1.csv', index=False)

In [ ]:
test1 = pd.read_csv('../input/mydata/test.csv')
test1

In [ ]:
transform_train = Compose([CenterCrop(200,200),HorizontalFlip(),
                           ShiftScaleRotate(),
                           RandomBrightnessContrast(),
                           ToTensorV2()])

transform_test = Compose([CenterCrop(200,200),
                           ToTensorV2()])

In [ ]:
train_dataset = IntracranialDataset(
    csv_file='train.csv', path=train_images_dir, transform=transform_train, labels=True)

valid_dataset = IntracranialDataset(
    csv_file='train.csv', path=train_images_dir, transform=transform_train, labels=True)

In [ ]:
valid_dataset = torch.utils.data.Subset(valid_dataset,range(0,134359))
print(len(valid_dataset))

train_dataset = torch.utils.data.Subset(train_dataset,range(134359, len(train_dataset)-1))
print(len(train_dataset))

In [ ]:
test_dataset = IntracranialDataset(
    csv_file='../input/mydata/test.csv', path=test_images_dir, transform=transform_test, labels=False)

In [ ]:
print(len(test_dataset))

In [ ]:
data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
print(len(data_loader_train))

data_loader_valid = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
print(len(data_loader_valid))

In [ ]:
data_loader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
print(len(data_loader_test))

In [ ]:
# Plot train example

batch = next(iter(data_loader_train))
fig, axs = plt.subplots(1, 5, figsize=(30,15))

for i in np.arange(5):

    axs[i].imshow(np.transpose(batch['image'][i].numpy(), (1,2,0))[:,:,0], cmap=plt.cm.bone)

In [ ]:
how_many_to_plot = 30

In [ ]:
plt.figure(figsize=(50,50))
for i, batch in enumerate(data_loader_train, start=1):
    
    plt.subplot(10,10,i)
    plt.imshow(np.transpose(batch['image'][i].numpy(), (1,2,0))[:,:,0], cmap='bone')
    plt.axis('off')
#     plt.title(train_set.classes[label.item()], fontsize=28)
    if (i >= how_many_to_plot): break
plt.show()

In [ ]:
#Importing the model
model_urls = {
    'resnext101_32x8d': 'https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth',
    'resnext101_32x16d': 'https://download.pytorch.org/models/ig_resnext101_32x16-c6f796b0.pth',
    'resnext101_32x32d': 'https://download.pytorch.org/models/ig_resnext101_32x32-e4b90b00.pth',
    'resnext101_32x48d': 'https://download.pytorch.org/models/ig_resnext101_32x48-3e41cc8a.pth',
}

In [ ]:
#Setting up the model
def _resnext(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
    model.load_state_dict(state_dict)
    return model

def resnext101_32x8d_wsl(progress=True, **kwargs):
    """Constructs a ResNeXt-101 32x8 model pre-trained on weakly-supervised data
    and finetuned on ImageNet from Figure 5 in
    `"Exploring the Limits of Weakly Supervised Pretraining" <https://arxiv.org/abs/1805.00932>`_
    Args:
        progress (bool): If True, displays a progress bar of the download to stderr.
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnext('resnext101_32x8d', Bottleneck, [3, 4, 23, 3], True, progress, **kwargs)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = resnext101_32x8d_wsl()
model.fc = torch.nn.Linear(2048,n_classes)

In [ ]:
#Loading state dictionaries
model = torch.nn.DataParallel(model)
checkpoint = torch.load('../input/models/png_model_e10_final.pt')
model.load_state_dict(checkpoint['model'])

In [ ]:
model.to(device)

In [ ]:
next(iter(data_loader_test))

In [ ]:
batch = next(iter(data_loader_test))
fig, axs = plt.subplots(1, 5, figsize=(15,5))

for i in np.arange(5):
    
    axs[i].imshow(np.transpose(batch['image'][i].numpy(), (1,2,0))[:,:,0], cmap=plt.cm.bone)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.eval()

test_pred = np.zeros((len(test_dataset) * n_classes, 1))

for i, x_batch in enumerate(tqdm.tqdm(data_loader_test)):
    
    x_batch = x_batch["image"]
    x_batch = x_batch.to(device, dtype=torch.float)
    
    with torch.no_grad():
        
        pred = model(x_batch)
        
        test_pred[(i * batch_size * n_classes):((i + 1) * batch_size * n_classes)] = torch.sigmoid(
            pred).detach().cpu().reshape((len(x_batch) * n_classes, 1))

# Testing with single image

In [ ]:
img1 = cv2.imread("../input/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x/ID_aec8e68b3.png")
# augmented = transform_test(image=img1)
sample_img = transform_test(image=img1)['image'].unsqueeze(0)
print(sample_img.shape)

In [ ]:
test_input = sample_img.to(device, dtype=torch.float)
print(test_input.shape)

In [ ]:
pred = model(test_input)

In [ ]:
pred

In [ ]:
proba = torch.sigmoid(pred).detach().cpu()
proba

In [ ]:
out_b = torch.round(proba)

In [ ]:
label_list = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']

In [ ]:
pred_lbl = np.where(out_b.float() == 1)[1]
pred_lbl

In [ ]:
label = []
for i in pred_lbl:
    label.append(label_list[i])
    
label

### Custom functions(WIP)

In [ ]:
def get_tensor(img):
    tfms_img = Compose([CenterCrop(200,200),
                           ToTensorV2()])
    img = cv2.imread(img)
    return tfms_img(image=img)['image'].unsqueeze(0)

In [ ]:
def predict(img, model, label_list):
    
    input_tensor = get_tensor(img)
    
#     if device == "cuda:0":
    input_tensor = input_tensor.to(device, dtype=torch.float)
#     else:
        #code for CPU device
        
    pred = model(input_tensor)
    
    #probabilities
#     if device == "cuda:0":
    proba = torch.sigmoid(pred).detach().cpu()
#     else:
        #code for CPU device
    
    #labels
    out = torch.round(proba)
    pred_lbl = np.where(out == 1)[1]
    
    label = []
    for i in pred_lbl:
        label.append(label_list[i])
        
    return proba, out, label

In [ ]:
#Testing function on sample image
predict('../input/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x/ID_ade010bbf.png', model, label_list)

# Visual explanations using Grad-CAM

In [ ]:
#importing libraries
import os
import random
import glob
import pandas as pd
import numpy as np
import pydicom 
from torch.hub import load_state_dict_from_url
from torchvision.models.resnet import ResNet, Bottleneck
import torch
import torch.optim as optim
from albumentations import Compose, ShiftScaleRotate, Resize, Normalize, HorizontalFlip, RandomBrightnessContrast,CenterCrop
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset,Subset
import cv2
#from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import roc_auc_score, classification_report, recall_score, f1_score, accuracy_score, precision_score, jaccard_score
from tqdm import notebook as tqdm

from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
dir_csv = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection'

train_images_dir = '../input/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x'
test_images_dir = '../input/rsna-test-stage-1-images-png-224x/stage_1_test_png_224x'

In [ ]:
#PARAMS
n_classes = 6
n_epochs = 10
batch_size = 32

COLS = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']

In [ ]:
transform_test = Compose([CenterCrop(200,200),
                           ToTensorV2()])

In [ ]:
#Importing the model
model_urls = {
    'resnext101_32x8d': 'https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth',
    'resnext101_32x16d': 'https://download.pytorch.org/models/ig_resnext101_32x16-c6f796b0.pth',
    'resnext101_32x32d': 'https://download.pytorch.org/models/ig_resnext101_32x32-e4b90b00.pth',
    'resnext101_32x48d': 'https://download.pytorch.org/models/ig_resnext101_32x48-3e41cc8a.pth',
}

In [ ]:
#Setting up the model
def _resnext(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
    model.load_state_dict(state_dict)
    return model

def resnext101_32x8d_wsl(progress=True, **kwargs):
    """Constructs a ResNeXt-101 32x8 model pre-trained on weakly-supervised data
    and finetuned on ImageNet from Figure 5 in
    `"Exploring the Limits of Weakly Supervised Pretraining" <https://arxiv.org/abs/1805.00932>`_
    Args:
        progress (bool): If True, displays a progress bar of the download to stderr.
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnext('resnext101_32x8d', Bottleneck, [3, 4, 23, 3], True, progress, **kwargs)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = resnext101_32x8d_wsl()
model.fc = torch.nn.Linear(2048,n_classes)

In [ ]:
#Loading state dictionaries
model = torch.nn.DataParallel(model)
checkpoint = torch.load('../input/models/png_model_e10_final.pt')
model.load_state_dict(checkpoint['model'])

In [ ]:
model.to(device)

In [ ]:
!pip install grad-cam

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
target_layers = [model.module.layer4[-1]]

In [ ]:
img1 = cv2.imread("../input/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x/ID_9b7d000a2.png")
# img1 = Image.open("../input/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x/ID_aec8e68b3.png")
print(img1.shape)
aug_img1 = transform_test(image=img1)
test_img = aug_img1['image'].unsqueeze(0)
print(test_img.shape)

In [ ]:
# img2 = np.array(test_img)
# img2 = Image.fromarray((img2 * 255).astype(np.uint8))
# img2

In [ ]:
img2.shape

In [ ]:
img2 = test_img.to(device, dtype=torch.float)
print(img2.shape)

In [ ]:
# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)

In [ ]:
grayscale_cam = cam(input_tensor=img2, targets=None)

In [ ]:
print(type(img1))
print(type(np.array(grayscale_cam)))

In [ ]:
cv2.imshow('CAM',grayscale_cam)

In [ ]:
grayscale_cam_np = np.array(grayscale_cam)
grayscale_cam_np.shape

In [ ]:
plt.imshow(grayscale_cam_np.reshape(200, 200, 1))

In [ ]:
cam = np.maximum(cam, 0)
cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))  # Normalize between 0-1
cam = np.uint8(cam * 255)  # Scale between 0-255 to visualize

In [ ]:
cam2 = Image.from_array(cam.traspose(1,2,0))

In [ ]:
alpha = 0.7

In [ ]:
overlay_img = Image.fromarray((alpha * np.asarray(img2) + (1 - alpha) * cam2).astype(np.uint8))

In [ ]:
plt.imshow(overlay_img)

In [ ]:
img1

In [ ]:
grayscale_cam

In [ ]:
!pip uninstall opencv-python-headless -y

In [ ]:
!pip install opencv-python --upgrade


In [ ]:
import cv2

In [ ]:
heatmap = cv2.applyColorMap(np.uint8(255 * grayscale_cam_np), cv2.COLOR_BG2RGB)

In [ ]:
 if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")

    cam = heatmap + img
    cam = cam / np.max(cam)

In [ ]:
visualization = show_cam_on_image(img1, grayscale_cam_np, use_rgb=True)